## Data from World Happiness Report

The World Happiness Report is an annual publication of the United Nations Sustainable Development Solutions Network. It contains articles, and rankings of national happiness based on respondent ratings of their own lives, which the report also correlates with various life factors.

In this notebook we will explore the happiness of different countries and the features associated.
The datasets that we will use are available in *Data*: **happiness2020.pkl** and **countries_info.csv**.

Although the features are self-explanatory, here a summary: 

**happiness2020.pkl**
* country: *Name of the country*
* happiness_score: *Happiness score*
* social_support: *Social support (mitigation the effects of inequality)*
* healthy_life_expectancy: *Healthy Life Expectancy*
* freedom_of_choices: *Freedom to make life choices*
* generosity: *Generosity (charity, volunteers)*
* perception_of_corruption: *Corruption Perception*
* world_region: *Area of the world of the country*

**countries_info.csv**
* country_name: *Name of the country*
* area: *Area in sq mi*
* population: *Number of people*
* literacy: *Literacy percentage*

In [ ]:
# For linux
#!head Data/countries_info.csv

In [ ]:
%load_ext nb_black

In [ ]:
import pandas as pd

%matplotlib inline

DATA_FOLDER = "Data/"

HAPPINESS_DATASET = DATA_FOLDER+"happiness2020.csv"
COUNTRIES_DATASET = DATA_FOLDER+"countries_info.csv"

## Task 1: Load the data

Load the 2 datasets in Pandas dataframes (called *happiness* and *countries*), and show the first rows.


**Hint**: Use the correct reader and verify the data has the expected format.

In [ ]:
# Write your code here
happiness = pd.read_pickle(HAPPINESS_DATASET)
countries = pd.read_csv(COUNTRIES_DATASET, decimal=",")

In [ ]:
happiness.head()

In [ ]:
happiness.info()
happiness.describe()

In [ ]:
countries.head()

In [ ]:
countries.info()
countries.describe()

## Task 2: Let's merge the data

Create a dataframe called *country_features* by merging *happiness* and *countries*. A row of this dataframe must describe all the features that we have about a country.

**Hint**: Verify to have all the rows in the final dataframe

In [ ]:
# Write your code here
country_features = pd.merge(
    happiness,
    countries,
    left_on=happiness["country"].str.lower(),
    right_on=countries["country_name"].str.lower(),
)
country_features = country_features.drop(columns=["key_0", "country_name"])

In [ ]:
country_features.head()

In [ ]:
country_features.info()
country_features.describe()

## Task 3: Where do people are happier?

Print the top 10 countries based on their happiness score (high is better).

In [ ]:
# Write your code here
happiness.nlargest(10, "happiness_score")["country"]

We are interested to know in what world region the people are happier. 

Create and print a dataframe with the (1) average happiness score and (2) the number of contries for each world region.
Sort the result to show the happiness ranking.

In [ ]:
df = pd.DataFrame({"A": ["a", "a"], "B": [1, 2], "C": [3, 4]})
df.groupby("A").agg(b=("B", lambda x: 0), c=("B", lambda x: 1))

In [ ]:
# Write your code here
regions = (
    happiness.groupby("world_region")
    .agg({"happiness_score": ["mean", "count"]})
    .sort_values(by=("happiness_score", "mean"), ascending=False)
)
regions.columns = ["happiness_mean", "count"]
regions

The first region has only a few countries! What are them and what is their score?

In [ ]:
# Write your code here
happiness[happiness["world_region"] == regions.iloc[0].name][
    ["country", "happiness_score"]
]

## Task 4: How literate is the world?

Print the name of countries with a level of literacy of 100%. 

For each country, print the name and the world region with the format: *{region name} - {country name} ({happiness score})*

In [ ]:
# Write your code here

literacy_100 = country_features[country_features["literacy"] == 100.0][
    ["country", "world_region", "happiness_score"]
]

print("Countries for which the literacy percentage is 100:")

for index, row in literacy_100.iterrows():
    region = row["world_region"]
    country = row["country"]
    score = row["happiness_score"]
    print(f"{region} - {country} ({score:.2f}%)")

What is the global average?

In [ ]:
# Write your code here
global_average = country_features["literacy"].mean()
print(f"The global(world) average is: {global_average:.2f}%")

Calculate the proportion of countries with a literacy level below 50%. Print the value in percentage, formatted with 2 decimals.

In [ ]:
# Write your code here
countries_less_50 = country_features[country_features["literacy"] <= 50.0]
proportion = len(countries_less_50) / len(country_features)
print(f"Proportion of countries with a literacy level below 50%: {proportion:.2f}")

Print the raw number and the percentage of world population that is illiterate.

In [ ]:
country_features[["population", "literacy"]]

In [ ]:
# Write your code here
num_illiterate = (
    country_features[["population", "literacy"]]
    .apply(lambda x: x["population"] * (1 - x["literacy"] / 100), axis=1)
    .sum()
)
percentage = num_illiterate / country_features["population"].sum()
print(
    f"Number of iliterate people: {num_illiterate:.0f}\nPercentage of world population that is illiterate: {percentage:.2f}"
)

## Task 5: Population density

Add to the dataframe a new field called *population_density* computed by dividing *population* by *area*.

In [ ]:
# Write your code here
country_features["population_density"] = (
    country_features["population"] / country_features["area"]
)

country_features[["country", "population", "area", "population_density"]].head()

What is the happiness score of the 3 countries with lowest population density?

In [ ]:
# Write your code here
country_features[["country", "population_density", "happiness_score"]].nsmallest(
    3, "population_density"
)

## Task 6: Healty and happy?

Plot in scatter plot the happiness score (x) and healty like expectancy (y).

In [ ]:
def make_plot(width, height):
    def assign_data(df):
        x = df["happiness_score"]
        y = df["healthy_life_expectancy"]
        z = df["literacy"]
        area = df["population"] / 2e5

        return x, y, z, area

    def add_labels_and_colorbar(fig, ax, sc):
        ax.set_xlabel("happiness_score")
        ax.set_ylabel("healthy_life_expectancy")
        cbar = fig.colorbar(sc)
        cbar.ax.set_ylabel("literacy")

    fig, ax = plt.subplots(figsize=(width, height))
    x, y, z, area = assign_data(country_features)
    sc = ax.scatter(x, y, c=z, s=area, alpha=0.5)
    add_labels_and_colorbar(fig, ax, sc)
    plt.show()

In [ ]:
# Write your code here
import matplotlib.pyplot as plt

make_plot(20, 10)

Feel free to continue the exploration of the dataset! We'll release the solutions next week.

----
Enjoy EPFL and be happy, next year Switzerland must be #1.